### NL2KQL (SLM) Replication Notebook (Gemma-3-4B w/Oracle):

The following notebook is a replication of [NL2KQL](https://arxiv.org/pdf/2404.02933) using Gemma 3 (4B). Gemma-3 4B is a powerful small language models developed by Google Deepmind. We choose Google Gemini 2.0 Flash as the oracle LLM, as this model provides free API access and strong inference capabilities. There are multiple main components involved in this pipeline:

- Schema Refiner
- Few-Shot Selector
- Prompt Builder

### Step #1: Import Necessary Packages:

The following packages are needed in order to run the pipeline. To install packages, you will need to run the following command in either your Terminal or in any of the cells:

```!pip install -r ../requirements.txt```

In [ ]:
# Import Necessary Packages:

import yaml
from google import genai as genai_client
from google.genai.types import GenerateContentConfig
from google.genai import types

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_huggingface import HuggingFacePipeline

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from huggingface_hub import login
import torch

from helpers import *

import pathlib
import textwrap
import time
import pandas
import json
import random
import asyncio
from collections import Counter

from IPython.display import display
from IPython.display import Markdown

import pickle
import warnings
warnings.filterwarnings('ignore')

with open("../config.yaml", "r") as f:
    token_config = yaml.safe_load(f)

In [ ]:
# Specify your HuggingFace Token here:
TOKEN = token_config['huggingface']['token']

# Replace this with your own Google token:
API_KEY = token_config['genai']['token']

# Path to prompt template:
prompt_template_path = ""

### Step #2: Creating the Few-Shot Embedding Store Database

The following box creates a Few-Shot Embedding Store Database. The purpose of this database is to create a variety of KQL examples that can be provided to the LLM in order to improve KQL code generation accuracy. Note that if an FSDB database has already been created, you do not need to run the block below (running the block below when an FSDB database exists will result in a message that says "FSDB already exists for Defender"). Instead run the second block below, which reads in the Defender FSDB database.

If you need to make any changes to how you generate the FSDB, see the ```helpers.py``` file.

NOTE: For the purposes of this project, we focus exclusively on Microsoft Defender for the time being.

In [ ]:
# themes = [
#     "Explore: Look for signs or hints of a security attack",
#     "Expansion: Searches for additional contextual understanding",
#     "Detect: Look for events related to a security attack",
#     "Remediate: Identify events for a given entity or asset",
#     "Report: Provide summary statistics for reporting"
# ]

# schema_file = "defender_fsdb_new.json"
# fsdb = generate_fsdb(themes, schema_file)

In [ ]:
with open('data/fsdb/defender_fsdb.json', 'r') as f:
    fsdb = json.load(f)

### Step #3: Generating the Table Embedding Store (Semantic Data Catalog)

As part of the Semantic Data Catalog, there are two different types of embeddings: Table Embeddings and Value Embeddings. In the next few steps, we will first make generate a table embedding dictionary and later build a value embedding dictionary to simulate the table embedding and value embedding stores discussed in NL2KQL:

Note: If a Table Embeddings file has already been created, then run the third block below instead to load the Table Embeddings directly from a json file:

In [ ]:
with open('data/miscellaneous/defender.yml', 'r') as file:
    defender_information = yaml.safe_load(file)
    
defender_embeddings = generate_table_embeddings(defender_information)

In [ ]:
with open('table_embeddings.json', 'w') as f:
    json.dump(defender_embeddings, f)

If you have already run the two code block snippets above before, then run the following block instead:

In [ ]:
with open('data/embeddings/defender_table_embeddings.json', 'r') as f:
    table_embeddings = json.load(f)

### Step #4: Generating the Value Embedding Stores (Semantic Data Catalog)

To preserve efficiency, we then find the embeddings of the columns of the filtered tables. Normally it would be efficient to store them all at once, but for this replication we just aim to generate the embeddings once we have the desired tables. We are bound by requests per day from the queries (1,500 per day) when querying Google Gemini 2.0, which is why we find the tables first, and then their respective embeddings.

Because the process takes a bit of time to generate, we do not supply the code here used to generate the embeddings (that is included in a separate notebook). Instead, just load the value embeddings through the block below:

In [ ]:
with open('data/embeddings/defender_value_embeddings.json', 'r') as f:
    value_embeddings = json.load(f)

### Step #5: Creating the Entire Pipeline

Now that we have the necessary components (Table Embeddings, Value Embeddings, Few-Shot Embeddings, we can build out the entire pipeline.

First we declare the evaluation dataset through the ```eval_df``` variable, and the ```queries``` variable to store all the Natural Language Queries (NLQs). Please note that if you wish to use a different evaluation dataset, you will need to change the code for ```eval_df``` and ```queries``` variables accordingly. All query results are stored in the ```df``` variable.

Furthermore, you will need to provide your Huggingface token under the ```TOKEN``` variable, as well as the Google API Key under the ```API_KEY``` variable. This is automatically provided through loading all tokens noted in the ```config.yaml```. Lastly, you will need to alter the ```iterations``` variable, depending on how many times you wish to run through the evaluation dataset. 

In [ ]:
eval_df = pd.read_json('data/evaluation/Defender_Evaluation.jsonl', lines=True)
queries= list(eval_df['context'])

df = pd.DataFrame()
query_count = 0

client = genai_client.Client(api_key=API_KEY)
iterations = 5

with open('data/DataCatalogs/Defender_DataCatalog.yml', 'r') as file:
    defender_catalog = yaml.safe_load(file)

The following snippet is used to store all intermediary responses from the SLM, final responses from the LLM Oracle, as well as all latency measures from the SLM and LLM Oracle.

In [ ]:
all_responses = []
all_nl2kql_txts = []
final_responses = []
slm_response_times = []
oracle_queries = []
oracle_response_times = []

oracle_token_count = 0
count = 0

model_name = ''
model_args = {
    "token": TOKEN,
    "dtype": torch.float32,
    "device_map": "auto",
}

model = AutoModelForCausalLM.from_pretrained(model_name, **model_args).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name, token=TOKEN)
pipe = transformers.pipeline("text-generation",
                             model=model,
                             tokenizer=tokenizer,
                             dtype=model_args["dtype"],
                             temperature=0.2,
                             do_sample=True)

In [ ]:
import re

def extract_inline_kql(text: str):
    if not isinstance(text, str):
        return None
    m = re.search(
        r"```(?:kusto|kql)\s*([\s\S]*?)```",
        text,
        flags=re.IGNORECASE
    )
    if m:
        return m.group(1).strip()
    return None

    

def norm(s: str) -> str:
    return s.replace("\r\n", "\n").replace("\r", "\n")

def extract_kql_from_result(result_text: str, prompt_text: str) -> str:
    """
    Strip the echoed prompt and return the FIRST ```kusto ...``` block.
    If no fenced block is found, return the remaining text.
    """
    full_norm = norm(result_text)
    prompt_norm = norm(prompt_text)

    # Strip prompt prefix if it was echoed
    if full_norm.startswith(prompt_norm):
        after_prompt = full_norm[len(prompt_norm):]
    else:
        header = "# KQL Generation Instructions"
        idx = full_norm.find(header)
        if idx != -1:
            after_prompt = full_norm[idx + len(header):]
        else:
            after_prompt = full_norm

    # Extract FIRST ```kusto ... ``` block
    m = re.search(r"```kusto\s*(.*?)```", after_prompt,
                  flags=re.DOTALL | re.IGNORECASE)
    if m:
        return m.group(1).strip()
    else:
        return after_prompt.strip()


for i in range(0, iterations):
        
    for query_prompt in queries:
    
        responses = []
        query_response = get_query_embedding(query_prompt)
        
        # Get the relevant tables to the query:
        defender_embedding_vals = list(table_embeddings.values())
        cosine_similarities = [cosine_similarity(np.array(query_response).reshape(1,-1), np.array(entry).reshape(1,-1)) for entry in defender_embedding_vals]
        
        cosine_similarities_vals = [float(entry) for entry in cosine_similarities]
        top_5_idx = np.argsort(cosine_similarities_vals)[-5:]
        table_lst = list(table_embeddings.keys())
        
        filtered_tables = []
        for idx in top_5_idx:
            filtered_tables.append(table_lst[idx])
        
        relevant_columns = dict()
        
        for k in filtered_tables:
            filter_lst = [entry for entry in list(value_embeddings.keys()) if k in entry]
            sub_dict = {key: value_embeddings[key] for key in filter_lst}
            cosine_similarities = []
            
            for key in sub_dict:
                cosine_similarity_val = cosine_similarity(np.array(value_embeddings[key]).reshape(1,-1), 
                                                          np.array(query_response).reshape(1,-1))
                cosine_similarities.append(cosine_similarity_val[0].item())
        
            top_5_cols_idx = np.argsort(cosine_similarities)[-5:]
        
            col_lst = list(sub_dict.keys())
            relevant_cols = []
            for idx in top_5_cols_idx:
                relevant_columns[col_lst[idx]] = cosine_similarities[idx]
        
        # Top 5 Values:
        final_vals = list(dict(sorted(relevant_columns.items(), key=lambda item: item[1], reverse=True)).keys())[0:5]
        final_vals_revised = []
        for entry in final_vals:
            try:
                final_vals_revised.append(re.search(r'Value Name:(.*)', entry).group(1))
            except:
                continue
            
        # Filter Few-Shot by Top t tables:
        filtered_fsdb = [entry for entry in fsdb if len(set(entry['tables']).intersection(set(filtered_tables))) > 0]
        
        # Semantic Similarity Matching:
        # nlq, f = 2
        
        fsdb_embeddings = []
        fsdb_count = 0
        for entry in filtered_fsdb:
            fsdb_response = client.models.embed_content(model = 'text-embedding-004',
                                                        contents = f"{entry['nlq']}")
        
            fsdb_embeddings.append({'NLQ': entry['nlq'], 'Embedding': fsdb_response.embeddings[0].values, 'KQL': entry['kql']})
            fsdb_count = fsdb_count + 1
            time.sleep(2)
            
        cosine_similarities_fsdb = [{'NLQ': entry['NLQ'], 'KQL': entry['KQL'], 'Similarity': float(cosine_similarity(np.array(query_response).reshape(1,-1), np.array(entry['Embedding']).reshape(1,-1)))} for entry in fsdb_embeddings]
        
        # Sort and find the Top 2 NLQ Entries:
        cosine_similarities_fsdb_sorted = sorted(cosine_similarities_fsdb, key=lambda x: x['Similarity'], reverse=True)[0:2]
        
        # Filter KQL Queries:
        cosine_similarities_fsdb_sorted = [{'NLQ': entry['NLQ'], 'KQL': entry['KQL'], 'Similarity': entry['Similarity']} for entry in cosine_similarities_fsdb_sorted]
        
        SCHEMA_PLACEHOLDER = ""
        EXAMPLES_PLACEHOLDER = ""
        USER_PLACEHOLDER = f"{query_prompt}"
        
        with open(prompt_template_path, 'r') as f:
            txt = f.read()
        
        # Add Table and Schema Information:
        for k in filtered_tables:
            temp_col_lst = []
            table = k
            
            for entry in defender_catalog:
                if entry['Name'] == table:
                    temp_col_lst = [subentry['Name'] for subentry in entry['Columns']]
                    
            #cols = relevant_columns[key]
            col_combined = ", ".join(temp_col_lst)
            SCHEMA_PLACEHOLDER += f"Table: {k}, Columns: {col_combined}"
            SCHEMA_PLACEHOLDER += "\n"
        
        # SCHEMA_PLACEHOLDER = "Tables: " + ", ".join(filtered_tables)
        txt = txt.replace('{{SCHEMA_PLACEHOLDER}}', SCHEMA_PLACEHOLDER)
        txt = txt.replace('{{USER_REQUEST_PLACEHOLDER}}', USER_PLACEHOLDER)
        
        # Add Examples:
        for entry in cosine_similarities_fsdb_sorted:
            EXAMPLES_PLACEHOLDER += f"NLQ: {entry['NLQ']} \n + KQL: {entry['KQL']}"
            EXAMPLES_PLACEHOLDER += "\n"
            
        txt = txt.replace('{{EXAMPLES_PLACEHOLDER}}', EXAMPLES_PLACEHOLDER)
    
        # Declare the SLMs:
        llmGeneratorOne = RunnableLambda(HuggingFacePipeline(pipeline=pipe))
    
        # Declare Langchain Runnable for Parallelism:
        runnable = RunnableParallel(result_one = llmGeneratorOne)
        
        # Perform the query:
        start = time.time()
        results = runnable.invoke(txt)
        end = time.time()
    
        slm_response_times.append(end-start)
    
        # Add main query:
        all_nl2kql_txts.append(txt)
        
        # Add all responses:
        # responses.append(re.search(r'model\\n(.*)', results['result_one'], flags=re.DOTALL).group(1))
        responses = []
        for key in ["result_one"]:
            full = results[key]
            kql_query = extract_kql_from_result(full, txt)
            responses.append(kql_query)

        # Store and debug-print
        all_responses.append(responses)
        for q in responses:
            print(q)      
        
        # all_responses.append(responses)
        # print(all_responses)

        #  Make changes as necessary to refine the best KQL query, and ensure that each column in the KQL query belongs to its respective table(s). You may use the following tables and columns:\n\n{schemas[0]}\n\n.


        try: 
            # Oracle Input can be determined:
            oracle_prompt = f"Given a Natural Language Query and a list of KQL queries, determine which of the following KQL queries is most syntactically and semantically correct:\n\nNatural Language Query: {query_prompt}\nResponses: {', '.join(responses)}.\n\nMake changes as necessary to refine the best KQL query, and ensure that each column in the KQL query belongs to its respective table(s)."           
            # f"Given a Natural Language Query and a list of KQL queries, determine which of the following KQL queries is most syntactically and semantically correct:\n\nNatural Language Query: {query_prompt}\nResponses: {', '.join(responses)}.\n\nMake changes as necessary to refine the best KQL query, and ensure that each column in the KQL query belongs to its respective table(s). You may use the following tables and columns: {SCHEMA_PLACEHOLDER}\n\nReturn the correct answer without explanation."
            oracle_start = time.time()
            oracle_response = client.models.generate_content(model = 'models/gemini-2.0-flash', contents = oracle_prompt)
            oracle_end = time.time()
            oracle_response_times.append(oracle_end - oracle_start)
        
            # Make changes as necessary to refine the best KQL query, and ensure that each column in the KQL query belongs to its respective table(s). You may use the following tables and columns:\n\n{schemas[0]}\n\n.
            oracle_queries.append(oracle_prompt)
            
            # Oracle Output can be determined:
            print(oracle_response.candidates[0].content.parts[0].text)
            final_responses.append(oracle_response.candidates[0].content.parts[0].text)
            count += 1
            print(count)
        except:
            time.sleep(60)
            oracle_prompt = f"Given a Natural Language Query and a list of KQL queries, determine which of the following KQL queries is most syntactically and semantically correct:\n\nNatural Language Query: {query_prompt}\nResponses: {', '.join(responses)}.\n\nMake changes as necessary to refine the best KQL query, and ensure that each column in the KQL query belongs to its respective table(s)."
            # f"Given a Natural Language Query and a list of KQL queries, determine which of the following KQL queries is most syntactically and semantically correct:\n\nNatural Language Query: {query_prompt}\nResponses: {', '.join(responses)}.\n\nMake changes as necessary to refine the best KQL query, and ensure that each column in the KQL query belongs to its respective table(s). You may use the following tables and columns: {SCHEMA_PLACEHOLDER}\n\nReturn the correct answer without explanation."            
            oracle_start = time.time()
            oracle_response = client.models.generate_content(model = 'models/gemini-2.0-flash', contents = oracle_prompt)
            oracle_end = time.time()
            oracle_response_times.append(oracle_end - oracle_start)
        
            # Make changes as necessary to refine the best KQL query, and ensure that each column in the KQL query belongs to its respective table(s). You may use the following tables and columns:\n\n{schemas[0]}\n\n.
            oracle_queries.append(oracle_prompt)
            
            # Oracle Output can be determined:
            print(oracle_response.candidates[0].content.parts[0].text)
            final_responses.append(oracle_response.candidates[0].content.parts[0].text)
            count += 1
            print(count)
            

### Step #6: Cost Analysis

An important aspect that we are tracking with these experiments is the amount of money it costs to run both SLMs and LLMs.

In [ ]:
all_queries = []
for i in range(0, iterations):
    all_queries = all_queries + queries

df = pd.DataFrame(columns = ['NLQ', 'LLM-KQL'])
df['NLQ'] = all_queries
df['LLM-KQL'] = final_responses

In [ ]:
from huggingface_hub import InferenceClient
from transformers import AutoTokenizer
HF_TOKEN = token_config['huggingface']['token']
SLM_MODEL_ID = "deepseek-ai/deepseek-coder-6.7b-instruct"

slm_client = InferenceClient(SLM_MODEL_ID, token=HF_TOKEN)
slm_tokenizer = AutoTokenizer.from_pretrained(SLM_MODEL_ID)
def slm_count_tokens(text: str) -> int:
    # add_special_tokens=False is usually closer to how APIs bill
    return len(slm_tokenizer.encode(text, add_special_tokens=False))


In [ ]:
slm_input_cost_per_million = 0.15
slm_output_cost_per_million = 0.15
oracle_input_cost_per_million = 0.10
oracle_output_cost_per_million = 0.40

total_slm_input_tokens = 0
total_slm_output_tokens = 0
total_oracle_input_tokens = 0
total_oracle_output_tokens = 0

# Calculate SLM Input tokens:
for entry in all_nl2kql_txts:
    total_slm_input_tokens +=slm_count_tokens(entry)

total_slm_input_tokens = total_slm_input_tokens
print("Finished calculating SLM input tokens")

# Calculate SLM Output tokens:
for sublst in all_responses:
    for resp in sublst:
        total_slm_output_tokens +=slm_count_tokens(resp)
print("Finished calculating SLM output tokens")

# Calculate Oracle input tokens:
for entry in oracle_queries:
    total_oracle_input_tokens = total_oracle_input_tokens + client.models.count_tokens(model="gemini-2.0-flash", contents=entry).total_tokens
print("Finished calculating Oracle input tokens")

# Calculate Oracle output tokens:
for result in final_responses:
    total_oracle_output_tokens = total_oracle_output_tokens + client.models.count_tokens(model="gemini-2.0-flash", contents=result).total_tokens
print("Finished calculating Oracle output tokens")

total_costs = ((slm_input_cost_per_million * total_slm_input_tokens)/1000000) + ((slm_output_cost_per_million * total_slm_output_tokens)/1000000) + ((oracle_input_cost_per_million * total_oracle_input_tokens)/1000000) + ((oracle_output_cost_per_million * total_oracle_output_tokens)/1000000)

In [ ]:
print("\n📊 SLM (Small Language Model)")
print(f"  Input tokens:  {total_slm_input_tokens:,}")
print(f"  Output tokens: {total_slm_output_tokens:,}")
print(f"  Total tokens:  {total_slm_input_tokens + total_slm_output_tokens:,}")

print("\n🔮 Oracle (Gemini 2.0 Flash)")
print(f"  Input tokens:  {total_oracle_input_tokens:,}")
print(f"  Output tokens: {total_oracle_output_tokens:,}")
print(f"  Total tokens:  {total_oracle_input_tokens + total_oracle_output_tokens:,}")

In [ ]:
print(f"Total Cost: ${round(total_costs/5, 3)}")
print(f"Average Latency (in sec): {round((sum(slm_response_times)/len(slm_response_times)) + (sum(oracle_response_times)/len(oracle_response_times)), 3)}")

### Step #7: Saving all Outputs

To keep track of all inputs and outputs utilized through the pipeline, we save the results from all variables in pickle files. This allows us to calculate more valuable information (i.e. Latency, Tokens used per query, etc.).

In [ ]:
# Replace the path as needed (i.e. Where the results will be stored):
base_path = "saleha-general-refinement-1"

with open(f"{base_path}all_responses.pkl", 'wb') as f:
    pickle.dump(all_responses, f)

with open(f"{base_path}all_nl2kql_txts.pkl", 'wb') as f:
    pickle.dump(all_nl2kql_txts, f)

with open(f"{base_path}final_responses.pkl", 'wb') as f:
    pickle.dump(final_responses, f)

with open(f"{base_path}oracle_queries.pkl", 'wb') as f:
    pickle.dump(oracle_queries, f)

with open(f"{base_path}slm_response_times.pkl", 'wb') as f:
    pickle.dump(slm_response_times, f)

with open(f"{base_path}oracle_response_times.pkl", 'wb') as f:
    pickle.dump(oracle_response_times, f)

# Save the main results, change the file path as needed (and RQ as needed):
df.to_csv(f"{base_path}rq5.csv")

### Step #8: Latency Measures:

Using the latency measures that we calculated from running the pipeline, we can calculate the total latency of our pipeline based on inference times.

In [ ]:
total_latency = np.array(slm_response_times) + np.array(oracle_response_times)
total_latency = total_latency.tolist()
print(sum(total_latency)/len(total_latency))

### Step #9: Export to YAML:

Next, we clean the results and export all of them to a .yaml file.

In [ ]:
print(eval_df.columns)
baseline = []
lst = []

# Change this string as needed (needs to be a .yaml):
path_to_file_export = ""

for i in range(0, iterations):
    baseline = baseline + list(eval_df['baseline']) 
df['baseline'] = baseline

for idx, row in df.iterrows():
    try:
        lst.append(re.search(r'```(?:kusto|kql)(.*)```', row['LLM-KQL'], flags=re.DOTALL).group(1))
    except:
        print(idx)
        lst.append(row['LLM-KQL'])

df['LLM-KQL-Extracted'] = lst

In [ ]:
# just for general refinement deepseek 

# import re
# import pandas as pd

# def extract_inline_kql(text: str):
#     if not isinstance(text, str):
#         return None
#     m = re.search(
#         r"```(?:kusto|kql)\s*([\s\S]*?)```",
#         text,
#         flags=re.IGNORECASE
#     )
#     if m:
#         return m.group(1).strip()
#     return None


# print(eval_df.columns)
# baseline = []

# # Change this string as needed (needs to be a .yaml):
# path_to_file_export = "saleha-general-refinement-1"

# # Repeat baseline for each iteration
# for i in range(0, iterations):
#     baseline = baseline + list(eval_df['baseline'])
# df['baseline'] = baseline

# # 1) apply extractor to get just the fenced KQL (or None)
# df["LLM_KQL_query_only"] = df["LLM-KQL"].apply(extract_inline_kql)

# # 2) if extractor failed (None/NaN), fall back to raw LLM-KQL
# df["LLM-KQL-Extracted"] = df["LLM_KQL_query_only"].where(
#     df["LLM_KQL_query_only"].notna(),  # condition
#     df["LLM-KQL"]                      # fallback
# )

# # 3) final safety: ensure everything is a string (no None in YAML)
# df["LLM-KQL-Extracted"] = df["LLM-KQL-Extracted"].astype(str)


In [ ]:
results = {"queries": []}

for idx, row in df.iterrows():
    results["queries"].append({
         "id": idx,
         "prompt": row["NLQ"],
         "connector": "Defender",
         "baseline": row["baseline"].strip(),
         "llmResult": row['LLM-KQL-Extracted']
})

with open(path_to_file_export, 'w') as f:
    yaml.dump(results, 
              f, 
              sort_keys=False,
              default_style='|',
              allow_unicode=True,
              width=1000)

### Step #10: Metrics

The following snippets will obtain all metrics for the results that you have just calculated. Please note that full path **must** be specified for ```file_of_interest``` and ```folder```. If you need help specifying the full path, use the ```!pwd``` command in Jupyter notebook.

In [ ]:
# DO NOT CHANGE THE FOLLOWING VARIABLE:
runner = "../offline_metrics_pipeline/offline-metrics-pipeline/offline_metrics_runner.py"

# CHANGE THE FOLLOWING VARIABLES:
# This should point to where your .yaml is currently stored (entire path must be specified):
file_of_interest = ""

# These should point to where you would like to store your results (entire path must be specified):
folder = ""
results_file = "testing.csv"

!python {runner} {file_of_interest} {folder} {results_file}